In [ ]:
pip install qutip

In [ ]:
from qutip import *
import copy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
vec1 = np.random.rand(5)+np.random.rand(5)*1j
vec2 = np.random.rand(5)+np.random.rand(5)*1j

In [ ]:
vec1

Useful functions in numpy: dot, vdot, inner. 



1.   Dot: Is the dot product of two 1D array vectors without complex conjugation $v^T v$.
2.   Vdot: Gives the dot product with complex conjugation $v^\dagger v$ or $\langle v | v \rangle $
3. inner: Has the ability to handle lists of vecots giving a matrix of dot products between the two lists. Note it does not have complex conjugation, just like Dot.


inner( [v1,v2] , [u1,u2] )  =\begin{bmatrix} v1^T u1 & v1^T u2\\
v2^T u1 & v2^T u2
\end{bmatrix}

In [ ]:
#some instructive tests with dot, vdot and inner. 

print('vdot of vec1 and vec1 is', np.vdot(vec1, vec1))
print('vdot of vec1 and vec2 is', np.vdot(vec1, vec2))
print('vdot of vec2 and vec1 is', np.vdot(vec2, vec1))
print('vdot of vec2 and vec2 is', np.vdot(vec2, vec2))
print('inner of vec1_dagger and vec2 is', np.inner(np.conj(vec1),vec2))
print('inner of list [vec1_dagger, vec2_dagger] and list [vec1,vec2] is', np.inner(np.conj([vec1, vec2]), [vec1, vec2]))

In [ ]:
# notice that without conjugation, the inner function gives wrong results for complex vectors.
np.inner([vec1, vec2], [vec1, vec2])

In [ ]:
# also notice the difference between dot and vdot. 
print('dot of v1 with itself', np.dot(vec1,vec1))

In [ ]:
vec_list = [vec1, vec2]

Let A denote the matrix whose columns are vectors of the set we are interested in. Then the projection opertor for this set of vectors can be created via. 

$$
P = A(A^† A)^{-1}A^†
$$

Note: In this formulation, the set of vectors need not be orthonormal. However, we do require the existance of inverse $(A^\dagger A)^{-1}$ which implies that the determinant of $A^\dagger A$ must be non-zero. 


Source: https://en.wikipedia.org/wiki/Projection_(linear_algebra)


Note: An assumption here is that the summation only involves linearly independent vectors.

In [ ]:
A = np.transpose(np.array(vec_list))
Adag = np.conj(np.transpose(A))

In [ ]:
print(A.shape,Adag.shape)

In [ ]:
AdagA = np.matmul(Adag, A)
AdagA

In [ ]:
invAdagA = np.linalg.inv(AdagA)
invAdagA

In [ ]:
np.matmul(AdagA, invAdagA) # test to check if the inverse has been computed correctly

In [ ]:
P_prime = np.matmul(invAdagA,Adag)
P = np.matmul(A,P_prime)
P.shape

In [ ]:
P2mP = (np.matmul(P,P)-P)
P2mP[np.abs(P2mP)>1e-15] # the resulting array is empty => P^2 is exactly as P, hence P is a projection operator

In [ ]:
Q = np.identity(5, 'complex') - P
Q.shape

In [ ]:
Q2mQ = (np.matmul(Q,Q)-Q)
Q2mQ[np.abs(Q2mQ)>1e-15] # the resulting array is empty => Q^2 is exactly as Q, hence Q is a projection operator

In [ ]:
w2 = np.diag([3, 2, 4,5, 8])**2
w2


In [ ]:
Pw2P = np.matmul(P,np.matmul(w2,P))
Pw2P.shape

In [ ]:
eval, evec = np.linalg.eig(Pw2P)

In [ ]:
eval[np.abs(eval)>0.0001]

In [ ]:
Ms_indices = np.where(np.abs(eval)>0.0001)[0]
Ms_indices

In [ ]:
Qw2Q = np.matmul(Q,np.matmul(w2,Q))
Qw2Q.shape

In [ ]:
evalQ, evecQ = np.linalg.eig(Qw2Q)
Mr_indices = np.where(np.abs(evalQ)>0.0001)[0]
Mr_indices

Basically the projection operator method is as follows. 

consider the diagonal Hessain matrix $\omega^2$ in the original phonon basis.
then 

$$
\omega^2 = I \omega^2 I = (P+Q) \omega^2 (P+Q) = (P\omega^2 P)+(Q\omega^2Q)+(P\omega^2Q)+(Q\omega^2P) = K + L
$$

In [ ]:
Ms = np.array([], 'complex')
Ms

In [ ]:
np.vstack((evec[:,1],evec[:,2]))


In [ ]:
Ms = np.transpose(evec[:,Ms_indices])
Ms.shape

In [ ]:
Mr = np.transpose(evecQ[:,Mr_indices])
Mr.shape

In [ ]:
L = np.matmul(P,np.matmul(w2,Q)) + np.matmul(Q,np.matmul(w2,P))

In [ ]:
# transforming L to the eigen basis of K
gammap = np.matmul(Ms,np.matmul(L,np.transpose(np.conj(Mr))))
gammap

In [ ]:
gammap.shape

Observation: it is best to transform your original hamiltonian such that the vectors g's are real values instead of complex.????

## question here is that should one use tranpose of Mr or transpose-conjugate of Mr.

In [ ]:
# now moving on to the electron-phonon couplings

vec1p = np.einsum('ij,j->i',Ms,vec1)
vec1p

In [ ]:
vec2p = np.einsum('ij,j->i',Ms,vec2)
vec2p

In [ ]:
Ap = np.matmul(Ms,A)

In [ ]:
Ap[:,0]

In [ ]:
U = np.transpose(np.vstack((Ms,Mr)))

In [ ]:
Udag = np.conj(np.transpose(U))

In [ ]:
hess = np.matmul(Udag,np.matmul(w2,U))
hess[np.abs(hess)>0.0001]

In [ ]:
eigvec = [Qobj(x) for x in np.transpose(U)]

In [ ]:
hess = Qobj(w2)
print(hess)

In [ ]:
hessp = hess.transform(eigvec)
print(hessp)

In [ ]:
w2p = np.matmul(U,np.matmul(hessp.full(),Udag))
print(Qobj(w2p).tidyup())

In [ ]:
gammap